# I record di C# 9
Sono indicati per descrivere dei _value type_, come gli importi monetari, le coordinate geografiche, gli indirizzi.



## Definire e istanziare un record
In C# 9 è stata introdotta l'apposita parola chiave `record`.  
Ecco la *sintassi posizionale*, molto concisa: i nomi tra parentesi sono i parametri che dovremo fornire al suo costruttore.

In [1]:
public record GeoPosition (float Latitude, float Longitude);

Ora che abbiamo definito il _record_ `GeoPosition`, andiamo a crearne un'istanza e visualizziamone il contenuto.

In [1]:
var position1 = new GeoPosition(41.9028f, 12.4964f); // Posizione di Roma
position1

Latitude,Longitude
41.9028,12.4964


Come si vede, il record `GeoPosition` ri-espone i valori che gli abbiamo fornito nel costruttore come proprietà `Latitude` e `Longitude`. Se vogliamo rendere più evidenti quali siano le proprietà presenti in un record, possiamo usare questa **sintassi alternativa**, che somiglia in tutto e per tutto a quella delle classi.


In [1]:
public record Address
{
    public string Street { get; init; }
    public string City { get; init; }
    public string Province { get; init; }
}

Creiamo anche un'istanza di questo record `Address`. In questo caso i valori andranno assegnati alle proprietà, durante l'inizializzazione dell'oggetto.

In [1]:
var address1 = new Address { Street = "Corso Mazzini, 44", City = "Ostia", Province = "RM" };
address1

Street,City,Province
"Corso Mazzini, 44",Ostia,RM


## Immutabilità
L'aver definito le proprietà `Street`, `City` e `Province` come `get; init;` ci permette di assegnare valori alle proprietà durante l'inizializzazione dell'oggetto e poi mai più.
 > Nota: La parola chiave `init` non è una prerogativa dei record, ma può essere usata anche nelle proprietà delle classi.

Ecco infatti che se provo a ri-assegnare un valore ad una proprietà dei record che abbiamo creato in precedenza, ottengo un errore. **Le proprietà sono quindi in sola lettura**, e queste istanze di record sono di fatto _immutabili_.

In [1]:
address1.Street = "Via Nazionale, 13";

Error: (1,1): error CS8852: L'indicizzatore o la proprietà di sola inizializzazione 'Address.Street' può essere assegnata solo in un inizializzatore di oggetto oppure in 'this' o 'base' in un costruttore di istanza o una funzione di accesso 'init'.

In [1]:
position1.Latitude = 42.9028f;

Error: (1,1): error CS8852: L'indicizzatore o la proprietà di sola inizializzazione 'GeoPosition.Latitude' può essere assegnata solo in un inizializzatore di oggetto oppure in 'this' o 'base' in un costruttore di istanza o una funzione di accesso 'init'.

Il concetto di immutabilità non è nuovo in C#, infatti poteva essere ottenuto anche in precedenza semplicemente fornendo i valori dell'oggetto come parametri del costruttore e ri-esponendoli come proprietà prive di `set`. Il tipo `DateTime` è un esempio di ciò.


In [1]:
var christmas = new DateTime(2020, 12, 25);
christmas.Month = 10;

Error: (2,1): error CS0200: Non è possibile assegnare un valore alla proprietà o all'indicizzatore 'DateTime.Month' perché è di sola lettura

Se ho bisogno di _mutare_, cioè modificare, il valore di una delle proprietà di un record sono quindi obbligato a creare una nuova istanza. Posso usare la nuova parola chiave `with` per _emendare_ un record esistente, cioè per modificare una sola delle sue proprietà mantenendo intatte le altre. In questo esempio ottengo un nuovo record `address2` modificando solo la proprietà `Street` del record `address1`.

In [1]:
Address address2 = address1 with { Street = "Via Nazionale, 13" };
address2

Street,City,Province
"Via Nazionale, 13",Ostia,RM


L'_immutabilità_ dei record è una caratteristica che ci aiuta a ridurre i bug nella nostra applicazione. Infatti, grazie ad essa abbiamo l'assoluta certezza che se passiamo l'istanza di un record a vari componenti della nostra applicazione, essi non potranno modificare i suoi valori in nessun modo.


## Estendibilità e comportamento
I record permettono di definire al loro interno ANCHE **metodi** e **proprietà con setter pubblico**. In questi casi i valori di un record potranno essere mutati.

In [1]:
public record Money
{
    public string Currency { get; init; } // Immutabile
    public decimal Amount { get; set; } // Mutabile
    
    public void Increment (decimal amount) { // Amount è mutabile anche con questo metodo
        Amount += amount;
    }
}

Ecco che infatti sono libero di modificare il valore di `Amount` anche dopo la costruzione dell'oggetto.

In [1]:
var price1 = new Money { Currency = "EUR", Amount = 1.00m };
price1.Amount = 2.00m;
price1.Increment(1.00m);
price1

Currency,Amount
EUR,3.00


Non mi è invece possibile modificare la proprietà `Currency` perché era stata definita come `get; init;`.

In [1]:
price1.Currency = "USD";

Error: (1,1): error CS8852: L'indicizzatore o la proprietà di sola inizializzazione 'Money.Currency' può essere assegnata solo in un inizializzatore di oggetto oppure in 'this' o 'base' in un costruttore di istanza o una funzione di accesso 'init'.

Anche se avessimo bisogno di mutare i valori, può comunque valere la pena di usare un record anziché una normale classe. Infatti, un'altra caratteristica importante dei _record_, che li differenza dalle classi, è che il compilatore li configura in modo che **l'uguaglianza tra due istanze venga calcolata membro a membro**, ovvero confrontando ciascuno dei valori delle loro proprietà.
 > Approfondimento tecnico: per far questo, il compilatore genera per noi un override del metodo `Equals` che è quello che viene chiamato quando confrontiamo due oggetti con gli operatori di confronto `==` e `!=`.

In [1]:
var total1 = new Money { Currency = "EUR", Amount = 15.00m };
var total2 = new Money { Currency = "EUR", Amount = 14.00m };
total2.Increment(1.00m);
$"Le due istanze sono uguali: {total1 == total2}"

Le due istanze sono uguali: True

I record **supportano anche l'ereditarietà**, proprio come le classi. Infatti, dietro le quinte il compilatore tratta i record proprio come fossero classi. Semplicemente, il compilatore configura i record per avere dei comportamenti particolari, come abbiamo visto nel caso precedente.

In [1]:
public record Investment : Money
{
    public int Years { get; init; }
}

Ed ecco che nel record `Investment` possiamo usare le proprietà e i metodi ereditati dal record base `Money`.

In [1]:
var investment = new Investment { Currency = "EUR", Amount = 100.00m, Years = 1 };
investment.Increment(50.00m);
investment

Years,Currency,Amount
1,EUR,150.00


Un'altra particolarità dei record che li differenzia dalle classi è che il compilatore ridefinisce anche il loro metodo `ToString()`. Proviamo a definire la stessa struttura dati sia come record che come classe e vediamo la differenza.

In [1]:
public record PointAsRecord
{
    public int X { get; init; }
    public int Y { get; init; }
}
public class PointAsClass
{
    public int X { get; init; }
    public int Y { get; init; }
}

var pointAsRecord = new PointAsRecord { X = 1, Y = 2 };
var pointAsClass = new PointAsClass { X = 1, Y = 2 };

In [1]:
pointAsRecord.ToString()

PointAsRecord { X = 1, Y = 2 }

In [1]:
pointAsClass.ToString()

Submission#41+PointAsClass

Come si vede dall'output, il metodo `ToString()` di un record riporta i valori contenuti in maniera molto chiara. Noi manteniamo comunque la facolta di ridefinire il metodo `ToString()` come preferiamo.

In [1]:
public record Metric
{
    public float Value { get; init; }
    public string Unit { get; init; }

    public override string ToString()
    {
        return $"{Value}{Unit}";
    }
}

In [1]:
var weight = new Metric { Value = 72.0f, Unit = "Kg" };
weight.ToString()

72Kg

## Serializzazione
I record possono facilmente essere **serializzati e deserializzati in formato json**, così che possano essere scambiati con altri sistemi informatici, ad esempio via REST API oppure salvati su disco o nel database.

In [1]:
using System.Text.Json;
var price3 = new Money { Currency = "EUR", Amount = 50.00m };
string serializedPrice = JsonSerializer.Serialize(price3);
serializedPrice

{"Currency":"EUR","Amount":50.00}

In [1]:
Money deserializedPrice = JsonSerializer.Deserialize<Money>(serializedPrice);
deserializedPrice

Currency,Amount
EUR,50.00


**Anche ASP.NET Core supporta in pieno i record**: possiamo infatti riceverli nei controller, come parametri delle action perché il model binding riesce a ricostruirli in base alle informazioni fornite con la richiesta, ad esempio tramite il post di un form.
In questo modo, ASP.NET Core su .NET 5 colma una lacuna importante: il model binding prima non era in grado di assegnare valori a proprietà prive di `set` pubblico, impedendoci così di lavorare con strutture _immutabili_.


## Decostruzione
Se usiamo la sintassi posizionale che abbiamo visto all'inzio, è molto facile **decostruire un record**, ovvero assegnare i suoi valori a singole variabili, rispettando il loro ordine. Decostruire un record è utile, ad esempio, quando vogliamo ottenere i valori da un record e assegnarli a variabili, così che possiamo mutarli a piacimento.
 > Nota: "decostruire" è a volte chiamato "destrutturare". Questi termini sono sinonimi e non hanno niente a che fare con il "distruggere" un oggetto. Infatti l'oggetto resta intatto e presente in memoria perché durante la decostruzione vengono soltanto letti i suoi valori e copiati su singole variabili.

In [1]:
public record Observation(float Azimuth, float Altitude);
var observation1 = new Observation(30.5511f, 77.14f);
var (azimuth, altitude) = observation1;
azimuth += 1.0f;
azimuth

31.5511

Se abbiamo definito il record usando la sintassi con le graffe, sarà necessario implementare esplicitamente la logica di decostruzione.
> La decostruzione è una funzionalità introdotta con C# 7.0 e perciò può essere usata anche con le classi.

In [1]:
public record Polar
{
    public double Angle { get; init; }
    public double Distance { get; init; }
    public void Deconstruct(out double angle, out double distance)
    {
        angle = Angle;
        distance = Distance;
    }
}

Possiamo indicare `_` per scartare le variabili che non ci interessa assegnare. In questo caso ci interessa ottenere solo il valore di `Distance`.

In [1]:
var polar1 = new Polar { Angle = Math.PI / 2, Distance = 13.5 };
var (_, distance) = polar1;
distance

13.5

## Per approfondire
 * Blog post Microsoft sui record: [https://devblogs.microsoft.com/dotnet/c-9-0-on-the-record/](https://devblogs.microsoft.com/dotnet/c-9-0-on-the-record/)
 * Comportamento del compilatore con i record: [https://www.thomasclaudiushuber.com/2020/09/01/c-9-0-records-work-with-immutable-data-classes/](https://www.thomasclaudiushuber.com/2020/09/01/c-9-0-records-work-with-immutable-data-classes/)